In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from gensim.parsing.preprocessing import PorterStemmer, remove_stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import WordCloud
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from textblob import TextBlob
import matplotlib.pyplot as plt
from six import string_types
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Sentiment Analysis using Rule Based Approach

When to use

if no labelled data available & if you have pos/neg lexicons for the language
When not to use

If lexicon list for your language, if it is not there
If text data is already processed, try avoiding

In [ ]:
reviews_data = pd.read_csv('../input/amazon-electronics-reviews/amazon_reviews_big.csv').sample(5000).reset_index()
reviews_data.head()

In [ ]:
reviews_data_df = pd.read_csv('../input/amazon-electronics-reviews/amazon_reviews_big.csv').sample(5000).reset_index()
reviews_data=reviews_data_df['reviewText'].fillna('Not available')
reviews_data

In [ ]:
def preprocess_data(reviews):
    all_reviews=[]
    lines=reviews.values.tolist()
    for document in lines:
        document=document.lower()
        document=re.sub(r"[,:;!@<>!~%$#*&//\\]"," ",document)
        tokens=word_tokenize(document)
        table=str.maketrans('','',string.punctuation)
        stripped=[w.translate(table) for w in tokens]
        word=[word for word in stripped if word.isalpha()]
        stop_words=set(stopwords.words('english'))
        lemmatizer=WordNetLemmatizer()
        words=[lemmatizer.lemmatize(w) for w in word if not w in stop_words]
        words=" ".join(words)
        all_reviews.append(words)
    return all_reviews
all_reviews=preprocess_data(reviews_data)
all_reviews[:2]

In [ ]:
df=pd.DataFrame()
df['reviewText']=reviews_data
df['processed_reviews']=all_reviews
df['Ratings']=reviews_data_df['overall']
df.head()

In [ ]:
df['Ratings'].value_counts()

In [ ]:
ml1=list(range(1,3))
mydict1=dict.fromkeys(ml1,'Negative')
ml2=list(range(3,4))
mydict2=dict.fromkeys(ml2,'Neutral')
ml3=list(range(4,6))
mydict3=dict.fromkeys(ml3,'Positive')
mydict1.update(mydict2)
mydict1.update(mydict3)
mydict1

In [ ]:
df['Ratings_Sentiment']=df['Ratings'].map(mydict1)
df.head()

In [ ]:
score=[round(TextBlob(text).sentiment.polarity,3) for text in all_reviews ]

In [ ]:
sentiment=['Positive' if scores >0 else 'Negative' if scores <0 else 'Neutral' for scores in score]

In [ ]:
df['Sentiments with TextBlob']=sentiment

In [ ]:
df.head()

In [ ]:
df['Sentiments with TextBlob'].value_counts()

# 1. VADER(Valence Aware Dictionary sEntiment Analyzer)

In [ ]:
# Do not perform lower case conversion
# Do not remove stop words
# Do not remove special characters
# Do not perform stemming or lemmatization

In [ ]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer=SentimentIntensityAnalyzer()
df['Sentiment_vader']=df['reviewText'].apply(lambda v:analyzer.polarity_scores(v)['compound'])
df.head()

In [ ]:
def get_sentiment_label_mod(compound_score):
    if compound_score>0.2:
        return 'Positive'
    elif compound_score<-0.2:
        return 'negative'
    else:
        return 'Neutral'
df['Sentiment_label_vader']=df['Sentiment_vader'].apply(get_sentiment_label_mod)
df.head()

In [ ]:
df['Sentiments with TextBlob'].value_counts()

In [ ]:
df['Sentiment_label_vader'].value_counts()

In [ ]:
df['Ratings_Sentiment'].value_counts()

# Based on Above three unsupervised approach we can interpret that Rating column is pretty much good indication to label the sentiment whcih we are validating using 2 rule based approach.

In [ ]:
df.head()

In [ ]:
from gensim.models import KeyedVectors, Word2Vec
embeddings = KeyedVectors.load_word2vec_format('../input/word2vec-google/GoogleNews-vectors-negative300.bin',binary=True)

In [ ]:
len(embeddings)

In [ ]:
df['Ratings_Sentiment_class'] = df['Ratings_Sentiment'].replace({'Negative': 0, 'Neutral': 1,'Positive':2}).astype(int)
df.head()

In [ ]:
docs = df['processed_reviews'].str.replace('<br />', '')
docs = docs.str.replace("n't", ' not').str.replace('[^a-zA-Z\s\.]', '')
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(docs, df['Ratings_Sentiment_class'],
                                                   test_size=0.2, random_state=1)

In [ ]:
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_x)
vocab_size = len(tokenizer.word_index) + 1
train_x_seq = tokenizer.texts_to_sequences(train_x)
test_x_seq = tokenizer.texts_to_sequences(test_x)

In [ ]:
train_x.shape

In [ ]:
train_x.iloc[0]

In [ ]:
tokenizer.word_index['really']

In [ ]:
train_x_seq[0]

In [ ]:
tokenizer.word_index['really']

In [ ]:
docs_size = []
for doc in train_x_seq:
    docs_size.append(len(doc))
pd.Series(docs_size).plot.box()

In [ ]:
maxlen = max(docs_size)
train_x_padded = pad_sequences(train_x_seq, padding='post', maxlen=maxlen)
test_x_padded = pad_sequences(test_x_seq, padding='post', maxlen=maxlen)

In [ ]:
maxlen

In [ ]:
vocab_size

In [ ]:
subset_embedding_matrix = np.zeros((vocab_size, 300))
words_available = []
words_not_available = []

for word, wid in tokenizer.word_index.items():
    if word in embeddings:
        subset_embedding_matrix[wid] = embeddings[word]
        words_available.append(word)
    else:
        words_not_available.append(word)

In [ ]:
subset_embedding_matrix

In [ ]:
len(words_not_available) / vocab_size * 100

In [ ]:
train_y

In [ ]:
test_y

In [ ]:
from tensorflow.keras.utils import to_categorical
Ytrain=to_categorical(train_y,num_classes=3)
Ytest=to_categorical(test_y,num_classes=3)
print('Ytrain shape is ',Ytrain.shape)
print('Ytest shape is ',Ytest.shape)

In [ ]:
train_x_padded.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten

In [ ]:
embedding_dim = 300
model = Sequential()
# trainable=False, if you have weight matrix already
model.add(layers.Embedding(vocab_size, embedding_dim,
                          weights = [subset_embedding_matrix],
                          input_length=maxlen,
                          trainable=False))
model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=3,activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
history = model.fit(train_x_padded, Ytrain, epochs=10, verbose=1)

In [ ]:
loss = model.evaluate(test_x_padded, Ytest)

In [ ]:
mytxt=pd.Series('really like case thing would like changed little cushion inside protect kindle like way clip kindle hold like real book fold cover back hold one handthe advise people buy know clipped onto hook way release push top hook took figure instruction crap')
test_seq=tokenizer.texts_to_sequences(mytxt)
test_seq_padded = pad_sequences(test_seq, padding='post', maxlen=maxlen)

In [ ]:
sentiment=np.argmax(model.predict(test_seq_padded))
print(sentiment)
if sentiment==0:
    print('Negative Review')
elif sentiment==1:
    print('Neutral Sentiment')
elif sentiment==2:
    print('Positive Sentiment')

# 2. Document Clustering

In [ ]:
# importing the dataset
reviews = pd.read_csv('../input/amazon-electronics-reviews/amazon_reviews_big.csv').sample(5000).reset_index()
reviews.head()

In [ ]:
reviews['reviewText'].isna().sum()

In [ ]:
docs = reviews['reviewText'].fillna('Not available').str.lower()
docs

In [ ]:
docs_string = ' '.join(docs)
stopwords = nltk.corpus.stopwords.words('english')
custom_stopwords = ['use', 'will', 'get', 'one', 'would', 'like']
all_stopwords = np.hstack([stopwords, custom_stopwords])
wc = WordCloud(background_color='white', stopwords=all_stopwords).generate(docs_string)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14,5))
plt.imshow(wc)

In [ ]:
docs = reviews['reviewText'].fillna('Not available').str.lower().str.replace('[^a-z\s]', '')
docs = docs.apply(remove_stopwords)
stemmer = PorterStemmer()
docs = stemmer.stem_documents(docs)

In [ ]:
vectorizer = TfidfVectorizer(min_df=5).fit(docs)
dtm = vectorizer.transform(docs)
vocab = vectorizer.get_feature_names()
df_dtm = pd.DataFrame(dtm.toarray(), columns=vocab)
df_dtm.shape

In [ ]:
from sklearn.cluster import KMeans

n_clusters = []
scores = []
for i in range(3, 10):
    n_clusters.append(i)
    kmeans_model = KMeans(n_clusters=i).fit(df_dtm)
    score = kmeans_model.inertia_
    scores.append(score)

In [ ]:
plt.plot(n_clusters, scores)

In [ ]:
kmeans = KMeans(n_clusters=7).fit(df_dtm)

In [ ]:
reviews['cluster'] = kmeans.labels_
reviews['cluster'].value_counts()

In [ ]:
for cluster_label in reviews['cluster'].unique():
    plt.figure(figsize=(14,5))
    cluster_reviews = reviews[reviews['cluster'] == cluster_label]['reviewText'].fillna('Not available')
    custom_stopwords = ['use', 'will', 'get', 'one', 'would', 'like']
    all_stopwords = np.hstack([stopwords, custom_stopwords])
    docs_string = ' '.join(cluster_reviews)
    wc = WordCloud(background_color='white', stopwords=all_stopwords).generate(docs_string)
    plt.imshow(wc)
    plt.title(cluster_label)